## Introduction

This notebook contains the code for implementing a GA with a modified encoding scheme instead of the one previously proposed in our group project proposal: values `0` to `19` for each of the 20 amino acids. The fitness function essentially evaluates the fitness using the PSSM generated as a result of iterating PSI-BLAST on UniProt/Swiss-Prot Manually Annotated dataset for 5 times.

MS2-L protein sequence retrieved from Uniprot:

<span style='color: gray;'>M</span>ETRFPQQSQQTPASTNRRRPFKHEDYPCRRQQRSST<span style='color: magenta;'>LYVLIFLAIF</span><span style='color: lime;'>LS</span><span style='color: cyan'>K</span>FTNQLLLSLLEAVIRTVTTLQQLLT

The sequence has been color-coded to represent key residues/motifs that are thought to be conserved and key to its function and its related **single gene lysis** protein relatives.

The sequence is also available for copying:
`METRFPQQSQQTPASTNRRRPFKHEDYPCRRQQRSSTLYVLIFLAIFLSKFTNQLLLSLLEAVIRTVTTLQQLLT`

## 1. Preliminaries

Herein the libraries and models in use are loaded and properly installed if needed.

In [ ]:
# UNUSED: Load this if you want to work with pygad (description above)
from pygad import GA

In [1]:
# Load this if you want to use my custom genetic algorithm class
import random as r
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# "chromosome" and GA class definition
class chromosome:
    def __init__(self, length = 75, numerical = True, possible_values = "ARNDCQEGHILKMFPSTWYV",
                 gene = "", custom_range : tuple[int] | None = None, pssm : dict = None, custom_init = None):
        self.gene = gene
        self.length = length
        self.numerical = numerical
        self.possible_values = possible_values
        self.custom_range = custom_range
        self.custom_init = custom_init
        self.pssm = pssm
        self.initialize()
    def initialize(self):
        if (self.numerical and len(self.values) <= 2):
            self.gene = ''.join([r.randint(0, 1)])
        else:
            if (self.pssm == None or self.custom_range == None):
                self.gene = ''.join(r.choice(self.possible_values))
            else:
                for pos in range(18, 28):
                    self.gene = ''.join([r.choices(population=self.possible_values, weights=self.pssm[pos])])
    def printChromosome(self):
        print(self.gene)
    def getChromosome(self):
        self.gene = str(self.gene)
    def __len__(self):
        return self.length

class GeneticAlgorithm():
    def __init__(self, setPop = 100, mutationRate = 4, generationNumber = 1, generationLimit = 100, selectionCoef = 0.5,
                 on_start = None, on_mutation = None, fitness = None):
        #setPop (the starting population) is defaulted out to 100, although it would be computationally heavy to process that number of chromosomes:
        self.setPop = setPop
        while self.setPop < 3:
            print("Error: Low starting population! Set a higher number:")
            self.setPop = int(input("New population size: "))
        self.fitness = self.on_start = self.on_mutation = None
        # Setting up custom functions should they exist
        if (fitness):
            self.fitness = fitness
        if (on_start):
            self.initializePop = on_start
        if (on_mutation):
            self.on_mutation = on_mutation
        # Initializing the population of solutions
        self.chrPop = []
        self.initializePop()
        # Setting up the GA instance parameters
        self.mutationRate = mutationRate
        self.generationNumber = generationNumber
        self.generationLimit = generationLimit
        self.selectionCoef = int(1/selectionCoef)
        self.solution_length = len(self.chrPop[0])

    def initializePop(self):
        self.chrPop = [chromosome().gene for x in range(self.setPop)]

    def fitnessScore(self, population, ReportMaxValue = True, ReportPopulationFitness = False):
        matchValue = []
        scoreValue = 0
        if (self.fitness):
            for chr in population:
                matchValue.append(self.fitness)
            scoreValue = (np.sum(matchValue) / len(population)) * 100
        for chr in population:
            chromosomeMatch = 0
            for x in range(len(chr)):
                if chr[x] == self.chrRef[x]:
                    chromosomeMatch += 1
            matchValue.append(chromosomeMatch / self.solution_length)
        scoreValue = (np.sum(matchValue) / len(self.chrPop))*100
        if ReportPopulationFitness == False:
            if ReportMaxValue == True:
                return max(matchValue)
            else:
                return(matchValue.index(max(matchValue)))
        else:
            return scoreValue

    def sortFunction(self):
        initialPopulation = self.chrPop
        fitPopulation = []
        for x in range(len(self.chrPop)):
            index = self.fitnessScore(initialPopulation, False)
            fitPopulation.append(initialPopulation[index])
            initialPopulation.pop(index)
        self.chrPop = fitPopulation

    def selectionFunction(self):
        # The default value for selectionCoef is 0.5, therefore self.selectionCoef equals 1/0.5 or 2, "selecting" only the chromosomes succeeding the 50th percentile:
        self.chrPop = self.chrPop[:len(self.chrPop) // self.selectionCoef]

    def crossover(self):
        # Defining indecies A and B for pairing random chromosomes chrA and chrB:
        for x in range(len(self.chrPop) // 2):
            self.fitnessScore(self.chrPop)
            A = r.randrange(len(self.chrPop))
            B = r.randrange(len(self.chrPop))
            parentA = self.chrPop[A]
            parentB = self.chrPop[B]
            parentAList, parentBList = list(parentA), list(parentB)
            crossoverPosition = r.randrange(self.solution_length)
            # Cutting and rejoining random segments on chrA and chrB (crossing over):
            offspring1, offspring2 = parentAList[:crossoverPosition], parentAList[crossoverPosition:]
            offspring1.extend(parentBList[crossoverPosition:])
            offspring2.extend(parentBList[:crossoverPosition])
            offspring1Chromosome, offspring2Chromosome = "", ""
            offspring1Chromosome = ''.join(offspring1)
            offspring2Chromosome = ''.join(offspring2)
            self.chrPop.append(offspring1Chromosome)
            self.chrPop.append(offspring2Chromosome)

    def mutation(self):
        for x in range(self.mutationRate):
            # Defining index A for mutating random chromosome chrA:
            A = r.randrange(len(self.chrPop))
            chrA = self.chrPop[A]
            chrAList = list(chrA)
            # Mutating chrA at a random position:
            chrAList[r.randrange(len(chrA))] = str(r.randint(0,1))
            chrA = ''.join(chrAList)
            self.chrPop[A] = chrA

    def printPopulation(self):
        print("Current population:", len(self.chrPop))
        for x in range(len(self.chrPop)):
            print("Chromosome #" + str(x+1) + ": " + str(self.chrPop[x]), end=";\n")
        index = self.fitnessScore(self.chrPop, False)
        print("The fittest chromosome:", self.chrPop[index])

    def runSimulation(self):
        print("Starting population fitness score:", self.fitnessScore(self.chrPop))
        print("Starting fitness score for the best chromosome:", self.fitnessScore(self.chrPop))
        print("Starting population:", self.setPop)
        xAxis = [self.generationNumber]
        yAxis = [self.fitnessScore(self.chrPop)]
        while self.generationNumber <= self.generationLimit:
            print("Fitness score for the best chromosome:", self.fitnessScore(self.chrPop))
            print("Generation #", self.generationNumber, sep="")
            print("Population fitness score:", self.fitnessScore(self.chrPop, False, True))
            # Appending more elements to x-axis and y-axis lists:
            xAxis.append(self.generationNumber + 1)
            yAxis.append(self.fitnessScore(self.chrPop))
            #self.printPopulation()
            # Sorting the solutions on the basis of their fitness
            self.sortFunction()
            self.selectionFunction()
            # Check whether on_mutation is not null
            self.mutation()
            # Perform crossover
            self.crossover()
            self.generationNumber += 1
        print("The resulting population:")
        self.printPopulation()
        print("The final fitness of generation Z:", self.fitnessScore(self.chrPop, False, True))
        print("The score for the fittest chromosome:", self.fitnessScore(self.chrPop))
        print("END OF SIMULATION.")
        # The final point:
        xAxis.append(self.generationNumber + 1)
        yAxis.append(self.fitnessScore(self.chrPop))
        plt.plot(xAxis, yAxis)
        plt.xlabel("Generations")
        plt.ylabel("Chromosome fitness")
        plt.title("Genetic Algorithm for Simulating and Selecting Individual Chromosomes (GASSIC)")
        plt.grid(True)
        plt.show()

    def fullySelectiveSimulation(self, threshold):
        print("Starting population fitness score:", self.fitnessScore(self.chrPop))
        print("Starting fitness score for the best chromosome:", self.fitnessScore(self.chrPop))
        print("Starting population:", self.setPop)
        while self.fitnessScore(self.chrPop, False, True) < threshold:
            print("Fitness score for the best chromosome:", self.fitnessScore(self.chrPop))
            print("Generation #", self.generationNumber, sep="")
            print("Population fitness score:", self.fitnessScore(self.chrPop, False, True))
            #self.printPopulation()
            self.sortFunction()
            self.selectionFunction()
            self.mutation()
            self.crossover()
            self.generationNumber += 1
        print("The resulting population:")
        self.printPopulation()
        print("The final fitness of generation Z:", self.fitnessScore(self.chrPop, False, True))
        print("The score for the fittest chromosome:", self.fitnessScore(self.chrPop))
        print("END OF SIMULATION.")

In [ ]:
# Generating a prediction for the initial sequence (using 2 recycle steps and 180 sampling steps)
!boltz predict "/workspaces/codespaces-jupyter/HTGAA/prediction_directory/HW5/initial_seq.fasta" --use_msa_server --recycling_steps 2 --sampling_steps 180 --accelerator cpu --out_dir "/workspaces/codespaces-jupyter/HTGAA/prediction_directory/HW5"

## 2. Defining Custom Functions

### 2.1. Parsing PSSM

In this part of the notebook, a PSSM for MS2-L (consisting of 75 residues for the matrices row-wise) will be loaded and parsed to create a position-specific list of amino acid weights. These weigths will be used in the `mutation_rule_for_ms2l` and `pssm_integration` functions as per the parameters set.

In [3]:
# Loading the PrettyTable library in order to visualize the PSSM
import prettytable

In [ ]:
def parse_pssm(file_path, query_len = 75):
    pssm = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for idl, line in enumerate(lines):
            if idl <= 2 or idl > query_len + 2:
                continue
            else:
                parts = line.split()
                # Turning the position and the scores into int
                position = int(parts[0])
                scores = list(map(float, parts[2:22]))
                # Normalizing the scores
                minScore = abs(min(scores))
                scores = [(score + minScore)/(20) for score in scores]
                pssm[position] = scores
    return pssm

# Usage
pssm_file = 'ms2l_psi_blast.pssm'
pssm_data = parse_pssm(pssm_file)
print(pssm_data)

{1: [0.1, 0.1, 0.05, 0.0, 0.05, 0.15, 0.05, 0.0, 0.05, 0.2, 0.25, 0.1, 0.45, 0.15, 0.0, 0.05, 0.1, 0.05, 0.1, 0.2], 2: [0.15, 0.2, 0.2, 0.3, 0.0, 0.3, 0.45, 0.1, 0.2, 0.05, 0.05, 0.25, 0.1, 0.05, 0.15, 0.2, 0.15, 0.05, 0.1, 0.05], 3: [0.1, 0.05, 0.1, 0.05, 0.1, 0.05, 0.05, 0.0, 0.0, 0.3, 0.2, 0.05, 0.2, 0.1, 0.05, 0.15, 0.3, 0.0, 0.05, 0.25], 4: [0.15, 0.5, 0.2, 0.1, 0.0, 0.25, 0.2, 0.1, 0.2, 0.05, 0.1, 0.3, 0.15, 0.05, 0.1, 0.15, 0.15, 0.05, 0.1, 0.05], 5: [0.1, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.1, 0.1, 0.15, 0.05, 0.15, 0.45, 0.0, 0.2, 0.1, 0.15, 0.25, 0.1], 6: [0.15, 0.1, 0.1, 0.1, 0.05, 0.15, 0.15, 0.1, 0.1, 0.1, 0.15, 0.15, 0.1, 0.05, 0.6, 0.15, 0.15, 0.0, 0.05, 0.1], 7: [0.1, 0.35, 0.15, 0.1, 0.0, 0.35, 0.2, 0.05, 0.15, 0.0, 0.05, 0.25, 0.1, 0.0, 0.05, 0.15, 0.1, 0.05, 0.05, 0.0], 8: [0.1, 0.2, 0.15, 0.1, 0.0, 0.4, 0.25, 0.05, 0.15, 0.0, 0.05, 0.2, 0.1, 0.0, 0.3, 0.15, 0.1, 0.05, 0.05, 0.05], 9: [0.2, 0.1, 0.15, 0.1, 0.1, 0.15, 0.15, 0.1, 0.1, 0.05, 0.05, 0.15, 0.1, 0.0

### 2.2. Creating the Functions

Here is the part of the sequence which does not overlap with neither the *cp* ORF nor the *rep* ORF:

(18)`RRRPFKHEDY`(27)

In [ ]:
# Here the sequence that is going to be used in the algorithm is included
seqRef = "METRFPQQSQQTPASTNRRRPFKHEDYPCRRQQRSSTLYVLIFLAIFLSKFTNQLLLSLLEAVIRTVTTLQQLLT"
seq = seqRef[17:27]
print(seq)

RRRPFKHEDY


In [ ]:
# Defining a derived class "proteinSeq"
class proteinSeq(chromosomes):
    def __init__(self):
        pass

In [ ]:
# Defining "chromosomes" for the genetic algorithm
# Each is a in fact a solution
# I have set the defaults according to the desired protein (i.e. MS2-L)

import sys
# This function takes in a parameter that
def initial_population(initial_seq : str, setPop = 10, possible_values = "", pssm = None):
    if pssm:
        pass
    else:
        pass

def mutation_rule_for_ms2l(gene : str | list[int] | list[str], current_range : tuple[int, int] = None, chosen_ranges : list[(int, int)] = None, pssm : dict = None):
    if (chosen_ranges):
        for seq_range in chosen_ranges:
            segment_to_mutate = gene[seq_range[0]:seq_range[1] + 1]
            if (pssm):
                pass # TO_BE_COMPLETED
    elif (current_range and pssm):
        for ind, pos in zip(range(len(gene)), range(current_range[0], current_range[1] + 1)):
            if type(gene) == str:
                gene = list(gene)
                gene[ind] = r.choices("ARNDCQEGHILKMFPSTWYV", pssm[pos])

def pssm_integration(gene : str | list[int], pssm_data : dict, current_range : tuple[int, int] = None):
    if (current_range):
        for ind, pos in zip(range(len(gene)), range(current_range[0], current_range[1] + 1)):
            if type(gene) == str:
                gene = list(gene)
                gene[ind] = r.choices("ARNDCQEGHILKMFPSTWYV", pssm_data[pos])

async def model_integration(gene, default_dir = "../HTGAA/prediction_directory/HW5/temp.fasta"):
    with open(default_dir, 'w') as f:
        f.write(gene)
    await sys('boltz predict "../HTGAA/prediction_directory/HW5/temp.fasta" --use_msa_server --recycling_steps 2 --sampling_steps 170 --accelerator gpu --out_dir "../HTGAA/prediction_directory/HW5"')


def fitness_function(gene : str | list[int], mode = "hybrid", pssm = None):
    if (mode == "hybrid"):
        pass
    elif (mode == "pssm"):
        pass
    elif (mode == "full"):
        pass
    else:
        pass


### 2.3. Running the algorithm

The functions previously defined in section 2.2 are assigned to an instance of the `GeneticAlgorithm` class.